# VinBigData Chest X-ray Abnormalities Detection
Automatically localize and classify thoracic abnormalities from chest radiographs

### The aim of this notebook is to demonstrate: 
1. Model Selection
2. Model Configuration
3. Model Training
4. Postprocessing and Inference
5. Model Evaluation
 
This notebook follows from the first [notebook](https://www.kaggle.com/bhallaakshit/dicom-wrangling-and-enhancement). The output from the first has been made available as a [dataset](https://www.kaggle.com/bhallaakshit/vinbig-tfrecords-for-object-detection?select=annotations) on Kaggle.
 
### Please consider giving an <font color="red">UPVOTE</font> if you find my work to be beneficial in any way. :D

## Install TF 2 Object Detection API
1. TF Model Garden
2. Protobuf
3. COCO API
4. Object Detection API 

In [ ]:
!# Download models
!git clone --depth 1 https://github.com/tensorflow/models

!# Compile proto files 
! # sudo apt install -y protobuf-compiler # Already present
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
%cd ..
%cd ..

!# Install cocoapi
!pip install cython 
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
%cd ..
%cd ..
!cp -r cocoapi/PythonAPI/pycocotools models/research/

!# Install object detection api
%cd models/research
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
%cd ..
%cd ..

## Import libraries
1. **Pandas**: Data manipulation
2. **Open-CV:** Computer Vision
3. **Matplotlib:** Plotting
4. **TensorFlow:** Deep Learning
5. **Miscellaneous**

In [ ]:
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from object_detection.utils import label_map_util as map_util
from object_detection.utils import visualization_utils as viz_util
from object_detection.utils import ops as ops_util
from object_detection.utils import config_util

import requests
import tarfile
from tqdm.notebook import tqdm
from io import BytesIO
from shutil import copy2
import random

Let's begin. Firstly, we set up our workspace.

In [ ]:
# Creating workspace
os.makedirs("workspace/pretrained_models", exist_ok = True)
os.makedirs("workspace/models", exist_ok = True)
os.makedirs("workspace/exported_models", exist_ok = True)
copy2("models/research/object_detection/model_main_tf2.py", "workspace")
copy2("models/research/object_detection/exporter_main_v2.py", "workspace")

## Reading TFRecords
Let's have a quick look at a sample record. Similar examples will be used for training.

In [ ]:
path_annot = "../input/vinbig-tfrecords-for-object-detection/annotations"
raw_dataset = tf.data.TFRecordDataset(os.path.join(path_annot, "annotations-00000-of-00025"))

for raw_record in raw_dataset.take(1): # Select one shard from the TFRecords dataset
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())

In [ ]:
def GetData(example):
    xmin = example.features.feature['image/object/bbox/xmin'].float_list.value
    xmax = example.features.feature['image/object/bbox/xmax'].float_list.value
    ymin = example.features.feature['image/object/bbox/ymin'].float_list.value
    ymax = example.features.feature['image/object/bbox/ymax'].float_list.value

    class_name_list = example.features.feature['image/object/class/text'].bytes_list.value
    class_name_list = [c.decode() for c in class_name_list]

    class_id_list = example.features.feature['image/object/class/label'].int64_list.value

    data = pd.DataFrame(
        zip(xmin, ymin, xmax, ymax, class_name_list, class_id_list), 
        columns = ["x_min", "y_min", "x_max", "y_max", "class_name", "class_id"]
    )

    height = example.features.feature['image/height'].int64_list.value[0]
    width = example.features.feature['image/width'].int64_list.value[0]

    data[["x_min", "x_max"]] = (data[["x_min", "x_max"]]*width).astype(int)
    data[["y_min", "y_max"]] = (data[["y_min", "y_max"]]*height).astype(int)

    LABEL_COLORS = [
        (230, 25, 75), (60, 180, 75), (255, 225, 25), (0, 130, 200), (245, 130, 48), (145, 30, 180), (70, 240, 240), 
        (240, 50, 230), (210, 245, 60), (250, 190, 212), (0, 128, 128), (220, 190, 255), (170, 110, 40), (255, 250, 200), 
    ]
    data["colors"] = data["class_id"].apply(lambda x: LABEL_COLORS[x])
    
    
    img_encoded = example.features.feature['image/encoded'].bytes_list.value[0]
    image = tf.io.decode_jpeg(img_encoded)
    
    return data, image

In [ ]:
data, image = GetData(example)

In [ ]:
%matplotlib inline

def plot_boxes(image, data, title):    
    img = cv2.cvtColor(image.numpy(), cv2.COLOR_GRAY2RGB)
    
    for i, row in data.iterrows():
    
        x1, y1 = row["x_min"], row["y_min"]
        x2, y2 = row["x_max"], row["y_max"]
    
        cv2.rectangle(
            img,
            pt1 = (x1, y1),
            pt2 = (x2, y2),
            color = row["colors"],
            thickness = 2
        )
    
        cv2.putText(
            img, 
            row["class_name"], 
            (x1, y1-5), 
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.5, 
            row["colors"], 
            1
        )

    plt.figure(figsize = (8, 8))
    plt.imshow(img) 
    plt.title(title)

plot_boxes(image, data, "Image extracted from TFRecord")

Great. Everything on track. Let's move on now.

## Model Selection
The TF 2 Object Detection API let's us play with SOTA object detection models pretrained on the Microsoft COCO [dataset](https://www.tensorflow.org/datasets/catalog/coco). These models have been made available as a GitHub [repository](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) called TensorFlow 2 Detection Model Zoo. We can fine-tune these models for our purposes and get great results.

The model of choice for this notebook is [EfficientDet](https://ai.googleblog.com/2020/04/efficientdet-towards-scalable-and.html). 

In [ ]:
# Download EfficientDet from Model Zoo
url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz"
path = "./workspace/pretrained_models"
r = requests.get(url)

# Extract model
thetarfile = tarfile.open(
    fileobj = BytesIO(r.content), 
    mode = "r|gz"
)

# Save model
thetarfile.extractall(path = path)

## Model configuration
The TensorFlow Object Detection API allows model configuration via the pipeline.config file that goes along with the pretrained model. The config file has 6 sections:
1. 'model'
2. 'train_config' 
3. 'train_input_config' 
4. 'eval_config'
5. 'eval_input_configs'
6. 'eval_input_config'

Not to be confused between 'eval_input_configs' and 'eval_input_config'. According to the [documentation](https://github.com/tensorflow/models/blob/c40b46ff63d1af2d32e6457dcb4a70d157648db2/research/object_detection/utils/config_util.py#L79):
> Keeps eval_input_config only for backwards compatibility. All clients should read eval_input_configs instead.

According to the official [tutorial](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#evaluation-sec), there are some basic changes to make to the config. We shall touch upon them only.

In [ ]:
# Moving pipeline.config file to models directory
fname = "pipeline.config"
model_name = "efficientdet_d0_coco17_tpu-32"

src = os.path.join(path, model_name, fname)
dst = src.replace("pretrained_", "").replace(fname, "")

os.makedirs(dst, exist_ok = True)

copy2(src, dst)

In [ ]:
path_label = "../input/vinbig-tfrecords-for-object-detection/LabelMap.pbtxt" 
LabelMap = map_util.create_category_index_from_labelmap(
    path_label, 
    use_display_name = True
)

In [ ]:
annot_dir = os.listdir(path_annot)
random.Random(0).shuffle(annot_dir)

train_data = annot_dir[:-4]
train_data = [os.path.join(path_annot, d) for d in train_data]

valid_data = annot_dir[-4:-2]
valid_data = [os.path.join(path_annot, d) for d in valid_data]

test_data = annot_dir[-2:]
test_data = [os.path.join(path_annot, d) for d in test_data]

In [ ]:
# Making recommended changes
fpath = os.path.join(dst, fname)
config_dic = config_util.get_configs_from_pipeline_file(fpath)

config_dic["model"].ssd.num_classes = len(LabelMap)
config_dic["model"].ssd.image_resizer.keep_aspect_ratio_resizer.min_dimension = 100

config_dic["train_config"].batch_size = 16
config_dic["train_config"].fine_tune_checkpoint = os.path.join(path, model_name, "checkpoint/ckpt-0")
config_dic["train_config"].fine_tune_checkpoint_type = "detection"
config_dic["train_config"].use_bfloat16 = False # Set to True if training on a TPU
config_dic["train_config"].num_steps = 1_000

config_dic["train_input_config"].label_map_path = path_label
config_dic["train_input_config"].tf_record_input_reader.input_path[:] = train_data

config_dic["eval_input_configs"][0].label_map_path = path_label
config_dic["eval_input_configs"][0].tf_record_input_reader.input_path[:] = valid_data

In [ ]:
# Save recommended changes
config = config_util.create_pipeline_proto_from_configs(config_dic)
config_util.save_pipeline_config(config, dst)

## Fine tuning object detection model (training)

In [ ]:
!python workspace/model_main_tf2.py --model_dir=$dst --pipeline_config_path=$fpath

## Exporting model

In [ ]:
!python workspace/exporter_main_v2.py --input_type=image_tensor --pipeline_config_path=$fpath --trained_checkpoint_dir=$dst --output_directory=workspace/exported_models/$model_name

In [ ]:
# To export model outside, first compress it
tar_model_name = model_name + ".tar.gz"
!tar -zcvf workspace/exported_models/$tar_model_name workspace/exported_models/$model_name

Handy method to download compressed file:
<a href="workspace/exported_models"> Click to Download </a>

## Postprocessing and Inference
Let's look at one sample x-ray. 

**IMPORTANT** 

TensorFlow expects input in NHWC format, which means: (batch-size, height, width, channels). Since our x-rays are grayscale, we can use OpenCV's cv2.COLOR_GRAY2RGB to solve the problem.

In [ ]:
for shard in test_data[:1]:
    raw_dataset = tf.data.TFRecordDataset(shard)
    
    for raw_record in raw_dataset.take(1): # Select one shard from the TFRecords dataset
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())

In [ ]:
img_encoded = example.features.feature['image/encoded'].bytes_list.value[0]
img = tf.io.decode_jpeg(img_encoded)
img = cv2.cvtColor(img.numpy(), cv2.COLOR_GRAY2RGB)
img = img[tf.newaxis, ...]

detector = tf.saved_model.load(os.path.join("workspace/exported_models", model_name, "saved_model"))
result = detector(img)

In [ ]:
result = {k:v.numpy() for k, v in result.items()}

In [ ]:
viz_util.visualize_boxes_and_labels_on_image_array(
    image = img[0], 
    boxes = result['detection_boxes'][0],
    classes = (result['detection_classes'][0]).astype(int), 
    scores = result['detection_scores'][0],
    category_index = LabelMap,
    use_normalized_coordinates = True,
    min_score_thresh = 0.4,
    line_thickness = 3,
    max_boxes_to_draw = 100,
)

In [ ]:
%matplotlib inline

plt.figure(figsize = (8, 8))
plt.imshow(img[0])
plt.title("Prediction")
plt.show()

## Evaluating performance
Let's look at what the real thoracic abnormalitites for this chest x-ray were.

In [ ]:
data, image = GetData(example)

In [ ]:
plot_boxes(image, data, "Ground Truth")

## Cross Validation and Hyperparameter Tuning
It would be amazing had there been a simple way to perform cross validation and hyperparameter tuning. Both involve iterative training consuming a lot of time and GPU (which may not always be available). Fortunately there are simple things that can be done to improve performance. For example, when making prediction, we can experiment with the confidence threshold (affecting classification) and tweak IoU (affecting localization). Impact on performance can be iteratively tested by comparing predictions against ground truth labels and bounding boxes, without retraining. This is a common strategy in ML and not included here.

### Please consider giving an <font color="red">UPVOTE</font> if you find my work to be beneficial in any way. :D